In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas, shapely
from pymongo import MongoClient,GEOSPHERE
from functools import partial

In [2]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

### Trying geo queries with differente collections

In [3]:
data = pd.read_csv("..\\data\\vivienda.csv")

In [5]:
data = data = geo_frame(data)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [7]:
test = data.sample(100)
coordinates = [list(i["coordinates"]) for i in test.geometry]

In [25]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

    return "Data succesfully added"

In [8]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [26]:
insert_object(data_test,h)

'Data succesfully added'

In [156]:
def lugar(coords, categ, radio):
    return list(p.find(
        {"category": categ, "geometry": {"$near": {
            "$geometry": {"type": "Point",
                          "coordinates": coords
                          }, "$maxDistance": radio}}}, {"name": 1, "longitude": 1, "latitude": 1, "category": 1}
    ))


def sum_gym(coords):
    result = coords, len(lugar(coords, "gym", 500))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_rest(coords):
    result = coords, len(lugar(coords, "restaurant", 500))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_night(coords):
    result = coords, len(lugar(coords, "nightlife", 5000))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_superm(coords):
    result = coords, len(lugar(coords, "supermarket", 1500))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_store(coords):
    result = coords, len(lugar(coords, "clothing_store", 2500))
    if result[1] > 10:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_medical(coords):
    result = coords, len(lugar(coords, "medical_centre", 6000))
    if result[1] > 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_transport(coords):
    result = coords, len(lugar(coords, "transport", 700))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_ent(coords):
    result = coords, len(lugar(coords, "general_entertainment", 5000))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_pharmacy(coords):
    result = coords, len(lugar(coords, "pharmacy", 500))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_parks(coords):
    result = coords, len(lugar(coords, "park", 1500))
    if result[1] > 2:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass

def sum_school(coords):
    result = coords, len(lugar(coords, "school", 5000))
    if result[1] > 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


In [67]:
len(list(filter(None, map(sum_gym,coordinates))))

43

In [10]:
coordinates = [list(i["coordinates"]) for i in test.geometry]

In [10]:
def category(choice):

    if choice == "Supermarkets":
        return list(filter(None, map(sum_superm, coordinates)))

    elif choice == "Gyms":
        return list(filter(None, map(sum_gym, coordinates)))

    elif choice == "Clothing store":
        return list(filter(None, map(sum_store, coordinates)))

    elif choice == "Nightlife":
        return list(filter(None, map(sum_night, coordinates)))

    elif choice == "Transport":
        return list(filter(None, map(sum_transport, coordinates)))

    elif choice == "Entertainment":
        return list(filter(None, map(sum_ent, coordinates)))

    elif choice == "Parks":
        return list(filter(None, map(sum_parks, coordinates)))

    elif choice == "Restaurants":
        return list(filter(None, map(sum_rest, coordinates)))

    elif choice == "Pharmacy":
        return list(filter(None, map(sum_pharmacy, coordinates)))

    elif choice == "Hospital":
        return list(filter(None, map(sum_medical, coordinates)))

    elif choice == "School":
        return list(filter(None, map(sum_school, coordinates)))


def final(district, cat_1, cat_2, cat_3, price):
    lista = []

    lista.append(category(cat_1))
    lista.append(category(cat_2))
    lista.append(category(cat_3))

    x = pd.DataFrame(lista[0])
    y = pd.DataFrame(lista[1])
    z = pd.DataFrame(lista[2])
    
    result = pd.concat([x,y,z])   
    
    result = result[result["district"] == district]
    
    if price == "Lower than 1K EUR":
        
        result = result[result["price"] <= 1000]
        
    elif price == "Between 1K and 2K EUR":
        
        result = result[(result.price > 1000) & (result.price <= 2000)]
    
    elif price == "Between 2K and 3K EUR":
        
        result = result[(result.price > 2000) & (result.price <= 3000)]
    
    elif price == "Between 3k and 4k EUR":
        
        result = result[(result.price > 3000) & (result.price <= 4000)]
        
    elif price == "Between 4k and 5k EUR":
        
        result = result = result[(result.price > 4000) & (result.price <= 5000)]
        
    elif price == "Higher than 5K EUR":
        
        result = result = result[(result.price > 5000)]
    
    if len(result) == 0:
        return "Your search had no resulls. If you want, try other parameters"
    else: 
        return result[result.groupby('latitude').latitude.transform('count')>2]

In [83]:
x = final("Centro", "Gyms", "Transport", "Hospital", "Between 2K and 3K EUR")

In [84]:
x = x.sort_values(by="latitude")

### Folium 

In [17]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [50]:
centre = 40.437863, -3.690433

In [18]:
def mapa(inicial_lat, inicial_long, zoom):
    
    x = folium.Map(location = [40.437863, -3.690433], zoom_start = 12.5)


def full_map(df):
    
    folium_map = folium.Map(location = [40.437863, -3.690433], zoom_start = 12.5)
    
    for i,row in df.iterrows():
        
        home = {
                "location" : [row["latitude"], row["longitude"]],
                "tooltip" : [row["price"], row["propertyType"],row["size"], row["url"]]
        }
        icon = Icon( color = "blue",
                        prefix = "fa",
                        icon = "home",
                        icon_color = "white"
        )
        Marker (**home,icon = icon).add_to(folium_map)
    return folium_map